<a href="https://colab.research.google.com/github/ijeoma91/Anomaly_Detection/blob/main/Copy_of_Batch_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
pip install pyspark

In [33]:
pip install spark

In [34]:
pip install cassandra-driver

In [35]:
pip install datetime

In [36]:
pip install timedelta

In [6]:
pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.0 MB/s eta 0:00:00


In [37]:
pip install pyngrok

In [38]:
! ngrok authtoken 2eWmBka8I8O6dJGkz6BO9XhP2hN_c7jsE4CwP8fjbqwidHdE
#from pyngrok import ngrok

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [39]:
import plotly.express as px
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import dcc
from dash import html
import dash_html_components as html

In [40]:

import os
packages = ["pandas", "schedule", "cassandra-driver"]
for packages in packages:
  os.system(f'pip install {packages}')

In [41]:
import pandas as pd
import datetime
#from datetime import datatime, timedelta
import time
import timedelta

In [42]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [43]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("My Data Batch Processing").getOrCreate()

# Replace 'dataset path' with the actual path to your CSV file
Batch_df = spark.read.csv("/content/iot_telemetry_data.csv", header=True, sep=',')

# Show the original DataFrame
print("Original DataFrame:")
Batch_df.show()

# Function to clean and uppercase column names
def clean_and_uppercase(name):
    # Remove unwanted characters (example: spaces and special characters)
    cleaned_name = ''.join(e for e in name if e.isalnum())
    # Make sure the first letter is uppercase
    cleaned_name = cleaned_name[0].upper() + cleaned_name[1:]
    return cleaned_name

# Rename columns
for col_name in Batch_df.columns:
    new_col_name = clean_and_uppercase(col_name)
    Batch_df = Batch_df.withColumnRenamed(col_name, new_col_name)

# Show the modified DataFrame
print("Modified DataFrame:")
Batch_df.show()

Original DataFrame:
+--------------------+--------------------+--------------------+---------+--------------------+----------+--------------------+--------------------+
|      "ts,""device""|              ""co""|        ""humidity""|""light""|             ""lpg""|""motion""|           ""smoke""|           ""temp"""|
+--------------------+--------------------+--------------------+---------+--------------------+----------+--------------------+--------------------+
|"1.59451209438597...|""0.0049559386483...|            ""51.0""|""false""|""0.0076508222705...| ""false""|""0.0204112701224...|           ""22.7"""|
|"1.59451209473556...|""0.0028400886071...|            ""76.0""|""false""|""0.0051143834009...| ""false""|""0.0132748367048...|""19.700000762939...|
|"1.59451209807357...|""0.0049760123404...|            ""50.9""|""false""|""0.0076732274063...| ""false""|""0.0204751255761...|           ""22.6"""|
|"1.59451209958914...|""0.0044030268296...|""76.800003051757...| ""true""|""0.00702333

In [44]:
# let's find how many device that has the highest temperature thereby grouping
from pyspark.sql.functions import *
from pyspark.sql.types import *


Batch_df.groupBy("Tsdevice").agg(countDistinct("Temp").alias("devices_count")).show()

+--------------------+-------------+
|            Tsdevice|devices_count|
+--------------------+-------------+
|"1.59452496005098...|            1|
|"1.59455160255505...|            1|
|"1.59482533195824...|            1|
|"1.59483897429002...|            1|
|"1.59487127495136...|            1|
|"1.59487128679406...|            1|
|"1.59453095631002...|            1|
|"1.59457600258627...|            1|
|"1.59457845300001...|            1|
|"1.59459310240455...|            1|
|"1.59465516904282...|            1|
|"1.59466583917450...|            1|
|"1.59483756707918...|            1|
|"1.59459084007783...|            1|
|"1.59455132218290...|            1|
|"1.59465414743181...|            1|
|"1.59478061339207...|            1|
|"1.59481121126203...|            1|
|"1.59483560211041...|            1|
|"1.59460478521667...|            1|
+--------------------+-------------+
only showing top 20 rows



In [45]:
from pyspark.sql import functions as F

# Assuming batch_df is your DataFrame and "Humidity" is the column you want to cast to float
Batch_df = Batch_df.withColumn("Humidity", F.col("Humidity").cast("float"))

# Show the result
Batch_df.show()

+--------------------+--------------------+--------+---------+--------------------+---------+--------------------+--------------------+
|            Tsdevice|                  Co|Humidity|    Light|                 Lpg|   Motion|               Smoke|                Temp|
+--------------------+--------------------+--------+---------+--------------------+---------+--------------------+--------------------+
|"1.59451209438597...|""0.0049559386483...|    NULL|""false""|""0.0076508222705...|""false""|""0.0204112701224...|           ""22.7"""|
|"1.59451209473556...|""0.0028400886071...|    NULL|""false""|""0.0051143834009...|""false""|""0.0132748367048...|""19.700000762939...|
|"1.59451209807357...|""0.0049760123404...|    NULL|""false""|""0.0076732274063...|""false""|""0.0204751255761...|           ""22.6"""|
|"1.59451209958914...|""0.0044030268296...|    NULL| ""true""|""0.0070233371458...|""false""|""0.0186282253770...|           ""27.0"""|
|"1.59451210176123...|""0.0049673636419...|    N

In [16]:
#initial_data(batch_df)

#spark.stop()

In [46]:
from pyspark.sql import SparkSession, functions as F
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

def initialize_cassandra_session():
    """
    Initializes and returns a Cassandra session.
    Ensure the Cassandra cluster is accessible through the specified host and port.
    """
    auth_provider = PlainTextAuthProvider(username='username', password='cassandra')
    cluster = Cluster(['0.tcp.eu.ngrok.io'], port=18718, auth_provider=auth_provider)
    session = cluster.connect()
    return session

def process_and_insert_data(dat, Batch_df):
    """
    Processes the provided DataFrame and inserts the aggregated data into Cassandra.
    """
    # Create aggregates
    agg_df = Batch_df.groupBy("Tsdevice").agg(
        F.avg('Co').alias('avg_co'),
        F.avg('Humidity').alias('avg_humidity'),
        F.avg(' Lpg').alias('avg_lpg'),
        F.avg('Smoke').alias('avg_smoke'),
        F.sum(F.when(F.col('Light') == 'true', 1).otherwise(0)).alias('light_true'),
        F.sum(F.when(F.col('Light') == 'false', 1).otherwise(0)).alias('light_false'),
        F.sum(F.when(F.col('Motion') == 'true', 1).otherwise(0)).alias('motion_true'),
        F.sum(F.when(F.col('Motion') == 'false', 1).otherwise(0)).alias('motion_false')
    ).collect()

    session = initialize_cassandra_session()
    session.execute("CREATE KEYSPACE IF NOT EXISTS batch WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3}")
    session.execute("""CREATE TABLE IF NOT EXISTS batch.sensors (
        device_id text PRIMARY KEY, date timestamp, Lpg float, Smoke float, Humidity float, Co float,
        light_true int, light_false int, motion_true int, motion_false int)""")

    # Insert data
    for row in agg_df:
        session.execute(
            """
            INSERT INTO batch.sensors (device_id, date, lpg, smoke, humidity, co, light_true, light_false, motion_true, motion_false)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (row['Tsdevice'], dat, row['avg_lpg'], row['avg_smoke'], row['avg_humidity'], row['avg_co'],
             row['light_true'], row['light_false'], row['motion_true'], row['motion_false'])
        )

    print("Data inserted into Cassandra for date:", dat)

def main(spark, Batch_df):
    # Example usage with a specific date
    process_and_insert_data('2024-04-02', Batch_df)  # Replace '2023-01-01' with your actual date variable



if Batch_df== "_main_":
    spark = SparkSession.builder.appName("My Data Batch Processing").getOrCreate()
    # Assuming you've loaded your DataFrame 'df' here
    # df = spark.read.csv("your_csv_file_path.csv", header=True, inferSchema=True)


    main(spark, Batch_df)  # Pass 'df' as an argument to 'main'

Phase 2 Feedback

Adding Data Validation and Robust error handling into the PySpark

Incorporting data validation and robust error handling into the PySpark code can significantly improve its reliability and maintainability, especially when dealing with large dataset and complex transformations.


### Data Validation

Data validation involves checking that the data meets certain criteria before processing it. This is curcial to prevent errors during execution and esure the quality of the outputs.

Firstly, Data Enhancement Validation

a. Schema Validation: Checking that the DataFrame contains the expected columns with the correct data types


In [47]:
from pyspark.sql.utils import AnalysisException

# Correct the dictionary to match the expected types
required_columns = {
    'Tsdevice': 'float',  # Corrected to 'float'
    'Co': 'float',
    'Humidity': 'float',
    'Lpg': 'float',
    'Smoke': 'float',
    'Light': 'string',
    'Motion': 'string'
}

def validate_schema(Batch_df, required_columns):
    try:
        for column, dtype in required_columns.items():
            # Validate if the DataFrame schema's data type matches the expected type
            if Batch_df.schema[column].dataType.simpleString() != dtype:
                # Raise an error if there's a mismatch
                raise ValueError(f"Column {column} must be of type {dtype}, found {Batch_df.schema[column].dataType.simpleString()}")
    except AnalysisException as e:
        # Collect all column names to indicate which are expected in the DataFrame
        missing_cols = ', '.join(required_columns.keys())
        raise ValueError(f"DataFrame is missing required columns: {missing_cols}") from e

# Example usage
#validate_schema(Batch_df, required_columns)

**### Data Completeness:**

Ensuring there are no missing values in critical columns before processing the Dataset

In [52]:
def check_missing_values(Batch_df, columns):
    for column in columns:
        missing_count = Batch_df.filter(F.col(column).isNull()).count()
        if missing_count > 0:
            raise ValueError(f"Column {column} contains {missing_count} missing values")

check_missing_values(Batch_df, ['Tsdevice', 'Co', 'Lpg', 'Smoke', 'Light', 'Motion'])

Step 2: Error Handling

Robust error handling ensures the application behaves predictably under various failure scenarios. This includes handling expections gracefully and possibly retrying operation if appropriate.

Example Enhancements for Error Handling:

1. Handling Database Connection and Query Failure
  a. We will implement retries for database operations.
  b. Use try-except blocks to handle and log error during database interations
  

In [50]:
import time

def execute_with_retries(session, query, parameters, max_retries=3):
    retry_count = 0
    while retry_count < max_retries:
        try:
            session.execute(query, parameters)
            break
        except Exception as e:
            retry_count += 1
            print(f"Failed to execute query, attempt {retry_count}/{max_retries}. Error: {e}")
            time.sleep(2)  # wait 2 seconds before retrying
            if retry_count == max_retries:
                raise

2. Wraping Processing in a Try-Except Block

This block can catch unexpected errors during data processing and log thm.

In [51]:
def process_and_insert_data(dat, Batch_df):
    try:
        validate_schema(Batch_df, required_columns)
        check_missing_values(Batch_df, ['Tsdevice', 'Co', 'Humidity', 'Lpg', 'Smoke', 'Light', 'Lotion'])
        agg_df = Batch_df.groupBy("Tsdevice").agg(...)
        session = initialize_cassandra_session()
        ...
        for index, row in pandas_df.iterrows():
            execute_with_retries(session, insert_query, (row['Tsdevice'], dat, ...))
        print("Data inserted into Cassandra for date:", dat)
    except Exception as e:
        print(f"Error processing data: {e}")
        raise

def main():
    try:
        spark = SparkSession.builder.appName("My Data Batch Processing").getOrCreate()
        ...
        process_and_insert_data('your_date_here', Batch_df)
    except Exception as general_error:
        print(f"An error occurred: {general_error}")